In [ ]:
import os, time, pandas as pd, time, re     # importar funciones externas
from Bio import Entrez, Medline
from tqdm import tqdm
from semanticscholar import SemanticScholar
import anexo_3_udfs as mf                   # importar funciones propias (UDFS)

print(os.getcwd())                          # comprobar WD
mf.viz()                                    # ampliar visualizaciones de pandas

# 1.cargar y de-duplicado

In [ ]:
# Cargar CSV de-duplicado
path1 = "./Anexo_3_carpeta_1_inputs/tabla_3.0_articulos_pre_revision.csv"
df = pd.read_csv(path1)
print(df.shape)
print(f"\n{df.columns}")

Añadir numero de duplicados a stats

### config para screening asistido

In [ ]:
# comprobar cuántos artículos con abstract tenemos
col = "Abstract Note"
nonan_values = df[col].value_counts(dropna=True)
nan_values = df[col].value_counts(dropna=False).iloc[0]

print(f"\nNo. papers: {len(df)}")
print(f"Papers w/ abstract in Zotero: {len(nonan_values)}")
print(f"Papers w/out abstract in Zotero: {nan_values}")

In [ ]:
# Preprocesamiento básico
df['combined_text'] = df['Title'].fillna('') + ' ' + df['Abstract Note'].fillna('')
df['combined_text'] = df['combined_text'].str.lower()

# Inicializar columna de decisiones
df['decision'] = None  # Posibles valores: 'include', 'exclude', None (pendiente)

# Modificación del preprocesamiento para manejar abstracts faltantes
df['Title'] = df['Title'].fillna('')
df['Abstract Note'] = df['Abstract Note'].fillna('')
df['has_abstract'] = ~df['Abstract Note'].str.strip().eq('')

# Crear una bandera para identificar registros sin abstract
print(f"Artículos sin abstract: {sum(~df['has_abstract'])} de {len(df)}")

# 2.recuperación automática de abstracts faltantes

## 2.1.recuperación automática de abstracts

In [ ]:
# Configuración para PubMed
mail = "pedro.fortes.gonzalez@sergas.es"
Entrez.email = mail

# Recuperar abstracts faltantes usando DOI
abstracts_recovered = 0
articles_without_doi = 0

# Crear una copia de los índices de artículos sin abstract
indices_without_abstract = df[~df['has_abstract']].index.tolist()

# Usar tqdm para mostrar progreso
for idx in tqdm(indices_without_abstract, desc="Recuperando abstracts"):
    doi = df.loc[idx, 'DOI']
    if pd.notnull(doi) and doi.strip() != '':
        abstract = mf.fetch_abstract_by_doi(doi, mail)
        if abstract and len(abstract.strip()) > 0:
            #df.at[idx, 'abstract'] = abstract
            df.at[idx, 'Abstract Note'] = abstract  # Actualizar también la columna original
            df.at[idx, 'has_abstract'] = True
            abstracts_recovered += 1
            time.sleep(0.5)  # Respetar límites de API
    else:
        articles_without_doi += 1

print(f"Abstracts recuperados: {abstracts_recovered}")
print(f"Artículos sin DOI: {articles_without_doi}")
print(f"Artículos que siguen sin abstract: {sum(~df['has_abstract'])}")

# exportar resultado
df.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.0_articulos_pre_revision.csv', index=False)

#### ¿cuántos abstracts hemos recuperado en automático?

In [ ]:
col = "has_abstract"
print(f"Nº artículos CON abstract:\n{df[col].value_counts(dropna=False)}")

#### 1/7
Voy a intentar recuperarlos manualmente. Para ello, primero separo el df en 2 (con y sin abstract) y los exporto para crear un checkpoint

#### Separar df en df_with_abstract y df_without_abstract
De acuerdo a columna "has_abstract"

In [ ]:
# Separar artículos con y sin abstract
df_with_abstract = df[df['has_abstract']].copy()
df_without_abstract = df[~df['has_abstract']].copy()

# Exportar lista de artículos CON abstract para revisión manual (si es necesaria) o recarga
df_with_abstract.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.1_articulos_con_abstract.csv', index=False)
print("Generado 'articulos_con_abstract.csv' para revisión manual")
print(f"Shape: {df_with_abstract.shape}")

# ídem pero para artículos SIN abstract
df_without_abstract.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.2_articulos_sin_abstract.csv', index=False)
print("Generado 'articulos_sin_abstract.csv' para revisión manual")
print(f"Shape: {df_without_abstract.shape}")

## 2.2.Recuperación manual de abstracts

Obtendré sus títulos y haré búsqueda avanzada en WOS y PubMed.

In [ ]:
# subsetear cols (para facilitar proceso)
cols = ['Author', 'Title', 'Publication Title', 'DOI', 'Abstract Note']
df_with_abstract = df_with_abstract[df_with_abstract.columns.intersection(cols)]
df_without_abstract = df_without_abstract[df_without_abstract.columns.intersection(cols)]

# almaceno títulos de los papers en lista
noabs_titles = df_without_abstract['Title'].value_counts(dropna=False).index.to_list()

# inicio las queries como string vacía
pm_query = ""
wos_query= ""
# defino variable de longitud total que quiero
total = len(noabs_titles)-1

***
#### query de PubMed

In [ ]:
# la construyo con un bucle
for i,t in enumerate(noabs_titles):
    if i == 0:                           # primer término
        pm_query = f"'{t}"
    elif i < total:                      # resto de términos (excp. último)
        pm_query += ("'[tiab] OR '" + t)
    elif i == total:                     # último término
        pm_query += ("'[tiab]")

# resultado
print(f"Total de títulos: {i+1}")
print("\nPubMed Title query:\n")
pm_query

Search with this query retrieved 0 results

***
#### query de WOS

In [ ]:
# la construyo con un bucle
for i,t in enumerate(noabs_titles):
    if i == 0:                           # primer término
        pm_query = f"TS=('{t}"
    elif i < total:                      # resto de términos (excp. último)
        pm_query += ("' OR '" + t)
    elif i == total:                     # último término
        pm_query += ("')")

# resultado
print(f"Total de títulos: {i+1}")
print("\nWOS Title query:\n")
pm_query

#### unir resultados con df

importar recuperados, hacer concat entre ellos (abs_rec) y pd.merge con df (df, abs_rec, left_join)

In [ ]:
# importar recuperados
#abs_rec_pm = pd.read_csv("tabla_3.2.1_abs_rec_pm.csv", sep=";")  # no lo ejecutamos ya que no recuperamos ninguno
abs_rec_wos = pd.read_csv("Anexo_3_carpeta_1_inputs/tabla_3.2.2_abs_rec_wos.csv", sep=";")

# seleccionar subset de columnas
pm_cols = ["AUTHOR", "TITLE", "JOURNAL", "DOI", "ABSTRACT"]
wos_cols = ['AU', 'TI', 'SO', "DL", "AB"]
abs_rec_wos = abs_rec_wos[abs_rec_wos.columns.intersection(wos_cols)]
abs_rec_wos = abs_rec_wos[sorted(abs_rec_wos)]

# renombrar cols
abs_rec_wos.columns = ["ABSTRACT", "AUTHOR", "DOI", "JOURNAL", "TITLE"]
# dropear duplicados
abs_rec_wos = abs_rec_wos.drop_duplicates()

# dropear nas en abstract
abs_rec_wos = abs_rec_wos.dropna(subset=["ABSTRACT"])

## merge both into rec_abs
rec_abs = abs_rec_wos.copy()
# drop articles w/out abstract & duplicates
rec_abs = rec_abs.drop_duplicates(subset=pm_cols[:-1])
rec_abs = rec_abs.dropna(subset=["ABSTRACT"])

# capitalizar columnas del df y renombrar "PUBLICATION TITLE" > "JOURNAL" para facilitar pd.merge()
df_with_abstract.columns = [col.upper() for col in list(df_with_abstract.columns)]
df_with_abstract.columns = ['AUTHOR', 'TITLE', 'JOURNAL',  'DOI', 'ABSTRACT']

## merge df & rec_abs
rec_abs = pd.concat([rec_abs, df_with_abstract])
rec_abs = rec_abs.drop_duplicates(subset="DOI")  # drop articles w/out abstract & duplicates
print(rec_abs.shape)                             # check

In [ ]:
# check
len_i = len(df_with_abstract)
print(f"Artículos iniciales c. abstract: {len_i}")
len_f = len(rec_abs)
recuperados = len_i - len_f
print(f"Artículos recuperados: {recuperados}")
print(f"Artículos finales c. abstract: {len_f}")

Hemos recuperado 0 artículos buscando por TÍTULO, busquemos por DOI

In [ ]:
# almaceno DOIS de los papers en lista
noabs_dois = df_without_abstract['DOI'].value_counts(dropna=False).index.to_list()
# print
for doi in noabs_dois:
    print(doi)

- En PM no obtengo nada (papers con otro DOI que dan como resultado este, y son Comentarios)
- En WOS sí que aparecen, pero son 5 Meeting Abstracts y 1 Letter --> los añado a excluidos

In [ ]:
# importar stats
stats = pd.read_csv('../Anexo_2_busqueda_bbdd/Anexo_2_carpeta_2_resultados/tabla_2.4_search_stats.csv', index_col=0)

# construir no_papers de deduplicaciones totales como dict (para luego concatenar)
temp_df = {"Not papers": [0, 0, len(noabs_dois), len_f]}
# pasar a df (y trasponer)
temp_df = pd.DataFrame(temp_df).T.astype(int)
# renombrar columnas
temp_df.columns = ["PubMed", "WOS", "Excluidos", "Total"]

# concatenar a stats
stats = pd.concat([stats, temp_df], axis=0)
stats

## 2.2.concatenar abstracts recuperados a articulos_con_abstract

In [ ]:
# como no hemos recuperado ningún abstract, podemos reasignar df y exportar
df = df[df["has_abstract"]==True]
print(f"Shape: {df.shape}")
df.head()

# 3.revisar manualmente abstracts no recuperados

In [ ]:
# añadir columnas checkbox
cols = ["ADPKD?", "OMICS?", "BIOINFO?", "HUMAN DATA?", "BIOMARKER?", "TRASLATIONAL?", "DECISION", "COMMENT"]
for col in cols[:-4]:
    df[col] = False
# últimas 2 cols
for col in cols[-4:]:
    df[col] = None

# check
df.head(2)

# 4.exportar todo
y empezar screening en siguiente script

In [ ]:
df.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.3.1_articulos_para_screening.csv', index=False)
print("Generado 'articulos_para_screening.csv' para screening")
print(f"Shape: {df.shape}")

stats.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.3.2.stats.csv', index=True)
print("Generado 'stats.csv' para screening")
print(f"Shape: {stats.shape}")